In [2]:
using JuMP, HiGHS
using Plots;
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
include("../src/utils.jl")
include("../src/unit_commitments.jl")
include("../src/plots.jl")
include("../src/network_analysis.jl")
ENV["COLUMNS"]=120; 

gen_df, gen_var_long, loads_long, network = process_data("../WECC")

zone_dict = Dict(zip(gen_df.zone, gen_df.zone_name));
line_dict = Dict(zip(network.network_lines, network."transmission path name"));

In [ ]:
# Create multiple scenarios with different conditions
# A spring day
n = 100
T_period = (n*24+6):(n*24+30)

# Scenario 1: Base case (normal conditions)
loads_scenario1 = loads_long[in.(loads_long.hour, Ref(T_period)), :]
gen_var_scenario1 = gen_var_long[in.(gen_var_long.hour, Ref(T_period)), :]

# Scenario 2: High demand scenario (10% increase)
loads_scenario2 = deepcopy(loads_scenario1)
loads_scenario2.demand .*= 1.10

gen_var_scenario2 = deepcopy(gen_var_scenario1)

# Scenario 3: Low renewable scenario (30% decrease in VRE)
loads_scenario3 = deepcopy(loads_scenario1)

gen_var_scenario3 = deepcopy(gen_var_scenario1)
gen_vre = gen_df[gen_df[!,:vre] .== 1, :r_id]
mask = in.(gen_var_scenario3.r_id, Ref(gen_vre))
gen_var_scenario3[mask, :cf] .*= 0.7  # 30% reduction

# Package scenarios
loads_scenarios = [loads_scenario1, loads_scenario2, loads_scenario3]
gen_var_scenarios = [gen_var_scenario1, gen_var_scenario2, gen_var_scenario3]

# Scenario weights (probabilities)
scenario_weights = [0.5, 0.3, 0.2]  # Base: 50%, High demand: 30%, Low renewable: 20%

println("Scenario 1 (Base): weight = $(scenario_weights[1])")
println("Scenario 2 (High Demand +10%): weight = $(scenario_weights[2])")
println("Scenario 3 (Low Renewable -30%): weight = $(scenario_weights[3])")
println("Total weight: $(sum(scenario_weights))")

In [ ]:
# Solve stochastic UC with robust commitment decisions
# Commitment decisions (COMMIT, START, SHUT) are made BEFORE knowing the scenario
# Operational decisions (GEN, FLOW, etc.) are made AFTER observing the scenario

stochastic_solution = stochastic_uc(
    gen_df, loads_scenarios, gen_var_scenarios, scenario_weights, network, 0.01
)

println("\nStochastic UC Solution Status: $(stochastic_solution.status)")
println("Expected Total Cost: \$$(round(stochastic_solution.cost, digits=2))")

In [ ]:
# Compare commitment decisions across scenarios
# Note: COMMIT is the same for all scenarios (first-stage decision)
# But GEN differs by scenario (second-stage decision)

println("\n=== Commitment Analysis ===")
println("Commitment decisions are IDENTICAL across all scenarios (robust decision)")
println("Generation decisions are DIFFERENT for each scenario (adaptive decision)\n")

# Show a sample thermal generator
sample_gen = gen_df[gen_df.up_time .> 0, :r_id][1]
println("Sample Generator ID: $sample_gen")
println("Technology: $(gen_df[gen_df.r_id .== sample_gen, :technology][1])")
println("\nCommitment status (same across scenarios):")
commit_values = stochastic_solution.commit[stochastic_solution.commit.r_id .== sample_gen, :]
println(first(commit_values, 10))

println("\nGeneration by scenario (differs by scenario):")
for s in 1:length(scenario_weights)
    gen_values = stochastic_solution.gen_scenarios[s]
    gen_sample = gen_values[gen_values.r_id .== sample_gen, :]
    println("\nScenario $s (weight = $(scenario_weights[s])):")
    println(first(gen_sample, 10))
end

In [ ]:
# Visualize generation across scenarios for a specific zone
zone = 1  # Pick a zone
zone_name = zone_dict[zone]

println("Analyzing zone: $zone_name")

# Prepare data for plotting
combined_data = DataFrame()

for s in 1:length(scenario_weights)
    # Get generation for this scenario
    gen_scenario = innerjoin(
        stochastic_solution.gen_scenarios[s], 
        gen_df[:, [:r_id, :technology, :zone]], 
        on = :r_id
    )
    
    # Filter to selected zone
    gen_zone = gen_scenario[gen_scenario.zone .== zone, :]
    
    # Aggregate by technology and hour
    gen_agg = combine(
        groupby(gen_zone, [:technology, :hour]), 
        :gen => sum => :gen_sum
    )
    
    # Add scenario identifier
    gen_agg[!, :scenario] .= "Scenario $s ($(Int(scenario_weights[s]*100))%)"
    gen_agg.hour = gen_agg.hour .- T_period[1]
    
    append!(combined_data, gen_agg)
end

# Plot using VegaLite
combined_data |>
@vlplot(
    columns = 3,
    facet = {field = "scenario", type = "nominal"},
    spec = {
        mark = :area,
        width = 300,
        height = 200,
        encoding = {
            x = {field = :hour, type = "quantitative", title = "Hour"},
            y = {field = :gen_sum, type = "quantitative", stack = :zero, title = "Generation (MW)"},
            color = {field = "technology", type = "nominal", scale = {scheme = "category10"}}
        }
    },
    title = "Generation by Technology in $zone_name Across Scenarios",
    resolve = {scale = {color = "independent"}}
)

In [ ]:
# Compare costs across scenarios
println("\n=== Cost Analysis ===")
println("Expected cost (weighted): \$$(round(stochastic_solution.cost, digits=2))")
println("\nCost breakdown by scenario:")

for s in 1:length(scenario_weights)
    # Calculate operational cost for this scenario
    gen_s = stochastic_solution.gen_scenarios[s]
    
    # Variable costs
    var_cost = sum(
        gen_df[gen_df.r_id .== row.r_id, :heat_rate_mmbtu_per_mwh][1] * 
        gen_df[gen_df.r_id .== row.r_id, :fuel_cost][1] * row.gen +
        gen_df[gen_df.r_id .== row.r_id, :var_om_cost_per_mwh][1] * row.gen
        for row in eachrow(gen_s) if gen_df[gen_df.r_id .== row.r_id, :vre][1] == 0
    )
    
    # VRE variable costs
    vre_cost = sum(
        gen_df[gen_df.r_id .== row.r_id, :var_om_cost_per_mwh][1] * row.gen
        for row in eachrow(gen_s) if gen_df[gen_df.r_id .== row.r_id, :vre][1] == 1
    )
    
    total_scenario_cost = var_cost + vre_cost
    weighted_cost = scenario_weights[s] * total_scenario_cost
    
    println("Scenario $s:")
    println("  Unweighted cost: \$$(round(total_scenario_cost, digits=2))")
    println("  Weighted cost:   \$$(round(weighted_cost, digits=2)) (weight = $(scenario_weights[s]))")
end

In [ ]:
# Analyze commitment decisions: which units are committed?
println("\n=== Commitment Summary ===")

# Count hours each generator is committed
commit_summary = combine(
    groupby(stochastic_solution.commit, :r_id),
    :commit => sum => :hours_committed
)

# Join with generator info
commit_summary = innerjoin(
    commit_summary,
    gen_df[:, [:r_id, :technology, :existing_cap_mw, :zone, :fuel]],
    on = :r_id
)

# Filter to show only units committed at least once
commit_summary = commit_summary[commit_summary.hours_committed .> 0, :]
sort!(commit_summary, :hours_committed, rev=true)

println("Top 10 most frequently committed units:")
println(first(commit_summary, 10))

println("\nCommitment by technology:")
tech_commit = combine(
    groupby(commit_summary, :technology),
    :hours_committed => sum => :total_hours,
    :r_id => length => :num_units
)
sort!(tech_commit, :total_hours, rev=true)
println(tech_commit)